### Format Preprocessing Codebook for Sarafu Trade Data 

Last edited: 18 March 2020

Author: Rebecca Mqamelo

Data: https://www.grassrootseconomics.org/research

**Introduction**

This notebook provides Python code for researchers who wish to re-format trade data taken from the Sarafu community currency network. The code below offers an example template for transforming publically available user and trade data into formats such as panel data, cross-sectional data and time-series data. 

In its current format, this notebook has been used to produce cross-sectional user data for analyzing the treatment effects of a randomized conrol trial on the Sarafu Network.

**Examples**

In the following scenarios, the researchers could use parts of this codebook to get the data format they need:

_1. Zenande wants to run a basic regression on the Sarafu network, but she has a unique list of dependent varibales that need to be calculated for every user at specific points in time._

_2. Senzo wants run a differences-in-differences analysis on a sample of treatment and control units in Nairobi. He has a list of outcome variables that we he wants to look at per user, but the current trade data does not account for these variables for every user, every day._

**Outline**

These are the basic steps in the data cleaning and format preprocessing pipeline:

1. Load .csv files as pandas DataFrames
2. Clean DataFrames to handle missing values, location errors and categorical inconsistencies.
3. Convert DataFrames into dictionary format (this significantly speeds up processing times for 500K+ rows of txnData and 40K+ rows of userData)
4. Loop through every user in the dictionary and use the txnData to compute time-specific user attributes like total trade volume, number of trade partners, items bought, etc. This is also an opportunity to explore custom user attributes for a richer dataset e.g. volume by trade category, trade frequency per trade category, etc.
5. Loop through the txnData dictionary and update user-specific attributes for each trading day – useful if you wish to apply methods like difference-in-differences or regression discontinuity design.
5. Once dictionaries are updated, convert them back into DataFrames and save in .csv format.

### Load Python libraries

In [84]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import stats
import seaborn as sns
from datetime import datetime
import random
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# Reduce decimal points to 2
pd.options.display.float_format = '{:,.2f}'.format

import statistics 
from statistics import mode 
from collections import Counter, OrderedDict
import os
import glob
import getopt
import sys
import math
import decimal
import copy
import csv
import progressbar
import time
from datetime import timedelta
import networkx as nx
import json 
import itertools
from collections import ChainMap

In [85]:
# Define the start and end dates for the trade data you are interesed in

start_date = datetime.strptime('2020-01-25', '%Y-%m-%d')

end_dates = [#datetime.strptime('2020-09-10', '%Y-%m-%d'), # 0: original baseline for selecting trt and ctrl
             datetime.strptime('2020-10-20', '%Y-%m-%d'), # 0: 1 month before 1st intervention
             datetime.strptime('2020-11-19', '%Y-%m-%d'), # 1: day before 1st intervention (analysis baseline)
             datetime.strptime('2020-12-11', '%Y-%m-%d'), # 2: one week after final transfer
             datetime.strptime('2020-12-18', '%Y-%m-%d'), # 3: two weeks after final transfer
             datetime.strptime('2021-01-04', '%Y-%m-%d'), # 4: one month after final transfer
             datetime.strptime('2021-02-04', '%Y-%m-%d'), # 5: two months after final transfer
             datetime.strptime('2021-03-04', '%Y-%m-%d')] # 6: three months after final transfer

In [34]:
def sample_from(dict_input):
    return {k: dict_input[k] for k in list(dict_input)[:2]}

### Read userData csv and save as dict

In [35]:
userData_raw = pd.read_csv('/Users/rebeccamqamelo/Desktop/RCT/Data/users_all_pub_20200125-20210316-all_time.csv', header=0, index_col=None)
userData_raw.head()

,id,start,final_bal,gender,area_name,area_type,held_roles,business_type,old_POA_blockchain_address,xDAI_blockchain_address,ovol_in,ovol_out,otxns_in,otxns_out,ounique_in,ounique_out,svol_in,svol_out,stxns_in,stxns_out,sunique_in,sunique_out
0,8505,2020-01-28 07:08:59.209905,0.00,NaN,other,other,ADMIN,System,NaN,0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0
1,24213,2020-06-08 08:39:43.913110,0.00,NaN,other,other,ADMIN,None,NaN,0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0
2,24208,2020-06-08 08:31:55.699422,0.00,NaN,other,other,ADMIN,None,NaN,0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0
3,24205,2020-06-08 08:13:46.215965,0.00,NaN,other,other,ADMIN,None,NaN,0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F,0.00,0.00,0,0,0,0,0.00,0.00,0,0,0,0
4,24212,2020-06-08 08:39:33.629046,0.00,NaN,other,other,ADMIN,None,NaN,0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F,0.00,"1,745,477.00",0,5740,0,4932,0.00,0.00,0,0,0,0


In [36]:
# Apply correct datetime format
userData_raw['start'] = pd.to_datetime(userData_raw['start'], format='%Y-%m-%d').dt.floor('d')
userData_raw.sort_values(by='start', inplace=True)
userData_raw.rename(columns={"final_bal": "bal"}, inplace=True)

# Group area names
userData_raw.loc[(userData_raw['area_name'] == 'Misc Nairobi'), 'area_name'] = 'Nairobi'
userData_raw.loc[(userData_raw['area_name'] == 'Mukuru Nairobi'), 'area_name'] = 'Nairobi'
userData_raw.loc[(userData_raw['area_name'] == 'Kinango Kwale'), 'area_name'] = 'Kwale'
userData_raw.loc[(userData_raw['area_name'] == 'Kilifi'), 'area_name'] = 'Kwale'

# Fix messy genders
nans = userData_raw.index[userData_raw['gender'].isnull()].tolist()
userData_raw.loc[(userData_raw.index.isin(nans)) & (userData_raw.held_roles != 'ADMIN'), 'gender'] = 'unknown gender'
userData_raw.loc[(userData_raw.index.isin(nans)) & (userData_raw.held_roles == 'ADMIN'), 'gender'] = 'system'
userData_raw.gender = userData_raw.gender.str.lower()
userData_raw.gender = userData_raw.gender.replace('male ', 'male')

userData_raw.to_csv('/Users/rebeccamqamelo/Desktop/RCT/Data/userData_raw_sorted.csv')

In [37]:
reader = csv.DictReader(open('/Users/rebeccamqamelo/Desktop/RCT/Data/userData_raw_sorted.csv'))

userData_dict = {}
for row in reader:
    key = row.pop('xDAI_blockchain_address')
    if key in userData_dict:
        # implement duplicate row handling here
        pass
    userData_dict[key] = row

In [38]:
# Check that the dictionary is in the correct format, with wallet addresses as keys
sample_from(userData_dict)

{'0xBAB77A20a757e8438DfaBF01D5F36DD12d862B31': OrderedDict([('', '46506'),
              ('id', '4'),
              ('start', '2018-10-21'),
              ('bal', '3633.88600155909'),
              ('gender', 'male'),
              ('area_name', 'Nairobi'),
              ('area_type', 'urban'),
              ('held_roles', 'BENEFICIARY'),
              ('business_type', 'Education'),
              ('old_POA_blockchain_address',
               '0xbe9e9a4b6bddec171bced854615b861479933e44'),
              ('ovol_in', '86624.7860015591'),
              ('ovol_out', '44348.0'),
              ('otxns_in', '23'),
              ('otxns_out', '6'),
              ('ounique_in', '2'),
              ('ounique_out', '1'),
              ('svol_in', '42979.1'),
              ('svol_out', '81622.0'),
              ('stxns_in', '69'),
              ('stxns_out', '73'),
              ('sunique_in', '39'),
              ('sunique_out', '39')]),
 '0xC1697C1326fD192438515fE2F7E4cCb0C705C5d2': OrderedDict([

### Read txnData csv and save as dict

In [39]:
txnData_raw = pd.read_csv('/Users/rebeccamqamelo/Desktop/RCT/Data/tx_all_pub_20200125-20210316-all_time.csv', header=0, index_col=None)
txnData_raw.head()

,id,timeset,transfer_subtype,source,target,weight,token_name,token_address
0,1,2020-01-25 19:13:17.731529,DISBURSEMENT,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,0x245fc81fe385450Dc0f4787668e47c903C00b0A1,"18,000.00",Sarafu,0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4
1,2,2020-01-25 19:13:19.056070,DISBURSEMENT,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,0xC1697C1326fD192438515fE2F7E4cCb0C705C5d2,"9,047.66",Sarafu,0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4
2,3,2020-01-25 19:13:20.288346,DISBURSEMENT,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,0xBAB77A20a757e8438DfaBF01D5F36DD12d862B31,"25,378.73",Sarafu,0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4
3,4,2020-01-25 19:13:21.478850,DISBURSEMENT,0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F,0xD95954e3fCd2f09A6Be5931D24f731eFa63BF435,"4,495.93",Sarafu,0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4
4,5,2020-01-26 07:48:43.042684,DISBURSEMENT,0xBDB3Bc887C3b70586BC25D04d89eC802b897fC5F,0x4AB73CfaC1732a9DcD74BdB4C9605f21832D7C72,400.00,Sarafu,0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4


In [40]:
# Apply correct datetime format
txnData_raw['timeset'] = pd.to_datetime(txnData_raw['timeset'], format='%Y-%m-%d').dt.floor('d')
txnData_raw.sort_values(by='timeset', inplace=True)

transfer_uses = []

for target in txnData_raw.target.values:
    transfer_uses.append(userData_dict[target]['business_type'])

txnData_raw['transfer_use'] = transfer_uses

txnData_raw.to_csv('/Users/rebeccamqamelo/Desktop/RCT/Data/txnData_raw_sorted.csv')

In [41]:
reader = csv.DictReader(open('/Users/rebeccamqamelo/Desktop/RCT/Data/txnData_raw_sorted.csv'))

txnData_dict = {}
for row in reader:
    date = row['timeset']
    sender_key = row['source']
    receiver_key = row['target']
    if sender_key in txnData_dict: # user's dict has already been created
        txnData_dict[sender_key].append({date: row})
    else:
        txnData_dict[sender_key] = [] # empty array to contain dict of each txn for that user
        txnData_dict[sender_key].append({date: row}) 
        
    if receiver_key in txnData_dict: # other user's dict has already been created
        txnData_dict[receiver_key].append({date: row})
    else:
        txnData_dict[receiver_key] = [] # empty array to contain dict of each txn for the other user
        txnData_dict[receiver_key].append({date: row}) 

In [42]:
for user in list(userData_dict.keys()):
    if(user in txnData_dict.keys()): # if user has participated in a trade in the txnData dict
        txns = txnData_dict[user] # list containing dicts for each txn
        for txn in txns: # parses each txn dict, where the key is the date 
            # (iteration therefore recognizes mutliple txns in a single day because it's going down a list
            date_str = list(txn.keys())[0]
            txn[date_str].update({'transfer_use': userData_dict[txn[date_str]['target']]['business_type']})

In [25]:
sample_from(txnData_dict)

{'0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F': [{'2020-01-25': OrderedDict([('',
                 '0'),
                ('id', '1'),
                ('timeset', '2020-01-25'),
                ('transfer_subtype', 'DISBURSEMENT'),
                ('source', '0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F'),
                ('target', '0x245fc81fe385450Dc0f4787668e47c903C00b0A1'),
                ('weight', '18000.0'),
                ('token_name', 'Sarafu'),
                ('token_address',
                 '0x0Fd6e8F2320C90e9D4b3A5bd888c4D556d20AbD4'),
                ('transfer_use', 'Savings Group')])},
  {'2020-01-25': OrderedDict([('', '1'),
                ('id', '2'),
                ('timeset', '2020-01-25'),
                ('transfer_subtype', 'DISBURSEMENT'),
                ('source', '0xEDA5C9B75Fdb3B9bdAB987A704632280Cf93084F'),
                ('target', '0xC1697C1326fD192438515fE2F7E4cCb0C705C5d2'),
                ('weight', '9047.660892016263'),
                ('token_n

### Update userData by time step

In [43]:
headersUserPub = ['xDAI_blockchain_address', 'start', 'gender', 'area_name', 'held_roles', 'business_type']

txHeaders = ['days_enrolled', 'days_active', 'bal', 'ovol_in', 'ovol_out', 'otxns_in', 'otxns_out', 'ounique_in', 
             'ounique_out', 'svol_in', 'svol_out', 'stxns_in', 'stxns_out', 'sunique_in', 'sunique_out', 
             'females_sold_to', 'females_bought_from', 'males_sold_to', 'males_bought_from', 
             'females_sold_to_vol', 'females_bought_from_vol', 'males_sold_to_vol', 'males_bought_from_vol', 
             'food_water_vol', 'labour_vol', 'health_vol', 'savings_vol', 'shop_vol', 
             'education_vol', 'transport_vol', 'energy_vol', 'environment_vol', 'other_vol', 'food_water_n', 
             'labour_n', 'health_n', 'savings_n', 'shop_n', 'education_n', 'transport_n', 'energy_n', 
             'environment_n', 'other_n']
    
for i in txHeaders:
    headersUserPub.append(i)

In [44]:
def create_supportNet(user, trans, supportNet, buddyTradeVol):
    t = trans
    if t['transfer_subtype'] == 'STANDARD':
        recipient_user_id = t['target']
        sender_user_id = t['source']
        
        otherId = sender_user_id
        if sender_user_id == user:  # if this user is the person sending the txn (outgoing trade)
            otherId = recipient_user_id

        volume = float(t['weight'])
        # A new addition to my support network
        if otherId not in buddyTradeVol.keys():
            buddyTradeVol[otherId] = volume
        else:
            buddyTradeVol[otherId] = buddyTradeVol[otherId] + volume

    supportNet[user] = buddyTradeVol
    
    return supportNet

In [46]:
def create_userData(userData, txnData, start_date, end_date, headersUserPub):
    
    supportNet = {user: {} for user in userData.keys()}
    
    for user in list(userData.keys()):
    
        bal = 0
        volume_in = 0
        volume_out = 0
        txns_in = 0 # all trades
        txns_out = 0
        unique_txns_in = 0 # unique trades
        unique_txns_out = 0
        svolume_in = 0
        svolume_out = 0
        stxns_out = 0
        stxns_in = 0
        sunique_txns_out = 0
        sunique_txns_out_group = 0
        sunique_txns_in = 0
        sunique_txns_out_atleast = 0
        stotal_unique_txns_out_atleast = 0
        stotal_unique_txns_out_atleast_group = 0
        sunique_txns_out_atleast_group = 0
        sunique_txns_in_atleast = 0
        females_sold_to = 0
        males_sold_to = 0
        females_bought_from = 0
        males_bought_from = 0
        females_sold_to_vol = 0
        males_sold_to_vol = 0
        females_bought_from_vol = 0
        males_bought_from_vol = 0

        sseenRecUsers = [] # user's recipients ("people who've bought from me")
        seenRecUsers = []

        sseenSentUsers = [] # recipient user's partner ("people I've bought from")
        seenSentUsers = []
        
        food_water_vol = 0
        labour_vol = 0
        health_vol = 0
        savings_vol = 0
        shop_vol = 0
        education_vol = 0
        transport_vol = 0
        energy_vol = 0
        environment_vol = 0
        other_vol = 0
        
        food_water_n = 0
        labour_n = 0
        health_n = 0
        savings_n = 0
        shop_n = 0
        education_n = 0
        transport_n = 0
        energy_n = 0
        environment_n = 0
        other_n = 0

        if(user in txnData.keys()): # if user has participated in a trade in the txnData dict
            
            txns = txnData[user] # list containing dicts for each txn
            
            # NB first txn is NOT 0-indexed because this would be their disbursement i.e registration txn
            # – this doesn't count as "active trading"
            if len(txns) == 1: 
                first_trade_date = datetime.strptime(list(txns[0].keys())[0], '%Y-%m-%d') 
            elif len(txns) > 1:
                first_trade_date = datetime.strptime(list(txns[1].keys())[0], '%Y-%m-%d') 
            last_trade_date = datetime.strptime(list(txns[-1].keys())[0], '%Y-%m-%d')
            start = datetime.strptime(userData[user]['start'], '%Y-%m-%d')
            end = end_date
            days_enrolled = (end - start).days
            days_active = abs(last_trade_date - first_trade_date).days
            
            #for trans in txnData[user]: # for each txn in the user's txn array
            
            for txn in txns: # parses each txn dict, where the key is the date 
                # (iteration therefore recognizes mutliple txns in a single day because it's going down a list
                date_str = list(txn.keys())[0]
                date = datetime.strptime(date_str, '%Y-%m-%d')
                if start_date <= date <= end_date: # checks that the date is within the defined range
                    
                    trans = txn[date_str]
                    supportNet = create_supportNet(user, trans, supportNet, {})
                    
                    if trans['source'] == user:
                        if trans['transfer_subtype'] != 'STANDARD':
                            volume_out+=float(trans['weight'])
                            txns_out+=1
                            if trans['target'] not in seenRecUsers:
                                seenRecUsers.append(trans['target'])
                                unique_txns_out+=1

                        else: # standard txn with other users
                            svolume_out+=float(trans['weight'])
                            stxns_out+=1
                            
                            if userData[trans['target']]['gender'] == 'female':
                                females_sold_to+=1
                                females_sold_to_vol+=float(trans['weight'])
                            elif userData[trans['target']]['gender'] == 'male':
                                males_sold_to+=1
                                males_sold_to_vol+=float(trans['weight'])
                            
                            if trans['target'] not in sseenRecUsers: # txn is standard and unqiue
                                sseenRecUsers.append(trans['target'])
                                sunique_txns_out+=1
                                if userData[user]['held_roles'] == "GROUP_ACCOUNT":
                                    sunique_txns_out_group += 1
                            
                            # What categories does this user spend their money on?
                            if trans['transfer_use'] == 'Food/Water':
                                food_water_vol+=float(trans['weight'])
                                food_water_n+=1
                            elif trans['transfer_use'] == 'Farming/Labour':
                                labour_vol+=float(trans['weight'])
                                labour_n+=1
                            elif trans['transfer_use'] == 'Health':
                                health_vol+=float(trans['weight'])
                                health_n+=1
                            elif trans['transfer_use'] == 'Savings Group':
                                savings_vol+=float(trans['weight'])
                                savings_n+=1
                            elif trans['transfer_use'] == 'Shop':
                                shop_vol+=float(trans['weight'])
                                shop_n+=1
                            elif trans['transfer_use'] == 'Education':
                                education_vol+=float(trans['weight'])
                                education_n+=1
                            elif trans['transfer_use'] == 'Transport':
                                transport_vol+=float(trans['weight'])
                                transport_n+=1
                            elif trans['transfer_use'] == 'Fuel/Energy':
                                energy_vol+=float(trans['weight'])
                                energy_n+=1
                            elif trans['transfer_use'] == 'Environment':
                                environment_vol+=float(trans['weight'])
                                environment_n+=1
                            elif trans['transfer_use'] == 'Other':
                                other_vol+=float(trans['weight'])
                                other_n+=1
                                
                    elif trans['target'] == user: # this txn marks where the user was a recipient
                        if trans['transfer_subtype'] != 'STANDARD':
                            volume_in+=float(trans['weight'])
                            bal+=float(trans['weight'])
                            txns_in+=1
                            
                            if trans['source'] not in seenSentUsers:
                                seenSentUsers.append(trans['source'])
                                unique_txns_in+=1
                                
                        elif trans['transfer_subtype'] == 'STANDARD': # standard txn with another user
                            svolume_in+=float(trans['weight'])
                            bal+=float(trans['weight'])
                            stxns_in+=1
                            
                            if userData[trans['source']]['gender'] == 'female':
                                females_bought_from+=1
                                females_bought_from_vol+=float(trans['weight'])
                            elif userData[trans['source']]['gender'] == 'male':
                                males_bought_from+=1
                                males_bought_from_vol+=float(trans['weight'])

                            
                            if trans['source'] not in sseenSentUsers:
                                sseenSentUsers.append(trans['source'])
                                sunique_txns_in+=1      
        
        data = [days_enrolled, days_active, bal, volume_in, volume_out, txns_in, txns_out, unique_txns_in, 
                unique_txns_out, svolume_in, svolume_out, stxns_in, stxns_out, sunique_txns_in, 
                sunique_txns_out, females_sold_to, females_bought_from, males_sold_to, males_bought_from, 
                females_sold_to_vol, females_bought_from_vol, males_sold_to_vol, males_bought_from_vol, 
                food_water_vol, labour_vol, health_vol, savings_vol, shop_vol, education_vol, transport_vol, 
                energy_vol, environment_vol, other_vol, food_water_n, labour_n, health_n, savings_n, shop_n, 
                education_n, transport_n, energy_n, environment_n, other_n]
        
    
        txData = {txHeaders[i]: data[i] for i in range(len(txHeaders))}
        
        uDict = userData[user]
        uDict.update(txData)
        userData[user] = uDict
        
    return userData, supportNet

In [47]:
def update_support_network():
    print(",.,.,.,.,.Calculating Support Network Strength")

    for tid in supportNet.keys():
        # This will be a running tally of the use's "carry over value"
        volOut = 0
        #print(supportNet[tid])

        # For each person in my support network...
        for bud in supportNet[tid].keys():

            spentOnBud = float(supportNet[tid][bud])
            volOutBud = float(userData[bud]['svol_out'])
            # This is a user's "carry over" effect expressed in volume
            if volOutBud >= spentOnBud:
                volOut += spentOnBud
            if volOutBud < spentOnBud:
                volOut += volOutBud 
                """
                # Actually, it might make more sense here to divide this number by the number of people in that 
                person's support network in proportion to the size of the trade those supporters had with this person.
                Yikes.
                """
                
                #look at the buddies of the buddy recursive
                #if bud in supportNet.keys():

        #supportRank[tid] = volOut

        tDict = userData[tid]
        if 'support_net' in list(userData[tid].keys()):
            tDict['support_net'] = volOut
        else:
            tDict.update({'support_net': volOut})
        userData[tid] = tDict

In [48]:
def generate_user_csv(txnData, userData, start_date, end_date, headersUserPub):
    
    headersUserPub.extend(['support_net'])
    #headersUser = headersUserPub
    filenameUser = '/Users/rebeccamqamelo/Desktop/Capstone/Data/userData' + start_date.strftime("%Y-%m-%d") + "-" + end_date.strftime("%Y-%m-%d") + '.csv'
    timestr = time.strftime("%Y-%m-%d")
    token_transactions = txnData
    indexR = 0
    seen_users = []
    exclude_list = []
    numberUsers = 0

    with open(filenameUser, 'w', newline='') as csvfileUser:

        spamwriterUser = csv.writer(csvfileUser)
        spamwriterUser.writerow(headersUserPub)

        for user_id, user_info in userData.items():

            user_data1 = {'xDAI_blockchain_address':  user_id}
            
            for i in headersUserPub:
                if i == 'xDAI_blockchain_address':
                    continue
                else:
                    user_data1[i] = user_info[i] 

            numberUsers+=1
            spamwriterUser.writerow([str(user_data1[k]) for k in headersUserPub])

    print("****saved all users to csv", filenameUser, " number of User:", numberUsers, timestr)

In [49]:
# This cell takes approx. 25 min to run! 8.52
for end_date in end_dates:
    print(end_date)
    # Refresh userData
    userData = userData_dict.copy()
    txnData = txnData_dict.copy()
    new_userData, supportNet = create_userData(userData, txnData, start_date, end_date, headersUserPub)
    update_support_network()
    generate_user_csv(txnData, new_userData, start_date, end_date, headersUserPub)

2020-10-20 00:00:00
,.,.,.,.,.Calculating Support Network Strength
****saved all users to csv /Users/rebeccamqamelo/Desktop/Capstone/Data/userData2020-01-25-2020-10-20.csv  number of User: 47375 2021-03-18
2020-11-19 00:00:00
,.,.,.,.,.Calculating Support Network Strength
****saved all users to csv /Users/rebeccamqamelo/Desktop/Capstone/Data/userData2020-01-25-2020-11-19.csv  number of User: 47375 2021-03-18
2020-12-11 00:00:00
,.,.,.,.,.Calculating Support Network Strength
****saved all users to csv /Users/rebeccamqamelo/Desktop/Capstone/Data/userData2020-01-25-2020-12-11.csv  number of User: 47375 2021-03-18
2020-12-18 00:00:00
,.,.,.,.,.Calculating Support Network Strength
****saved all users to csv /Users/rebeccamqamelo/Desktop/Capstone/Data/userData2020-01-25-2020-12-18.csv  number of User: 47375 2021-03-18
2021-01-04 00:00:00
,.,.,.,.,.Calculating Support Network Strength
****saved all users to csv /Users/rebeccamqamelo/Desktop/Capstone/Data/userData2020-01-25-2021-01-04.csv  num

### Format userData for regression

Load the pre-assigned treatment and control group wallet addresses and update the formatted userData dfs (for each intervention timestamp) with this new column. This gives a static time snapshot of user info at specific dates, useful for basic regressions.

_Note: We're taking the old df that contains the trt and ctrl groups for the study and updating them with the correct userData info (previous code has some errors, which is why the previous df is not relevent)_

In [50]:
trt_original = pd.read_csv('/Users/rebeccamqamelo/Desktop/Capstone/Original/trt_wallets_Will.csv', header=0, index_col=None)

Nairobi = pd.read_csv(
    '/Users/rebeccamqamelo/Desktop/Capstone/Original/Rebecca_Nairobi_sample_2x_per_week_filter.csv', # was actually a 1x week filter
    header=0, index_col=None)
print('Len Nairobi original sample:', len(Nairobi))
Kwale = pd.read_csv(
    '/Users/rebeccamqamelo/Desktop/Capstone/Original/Rebecca_Kwale_sample_2x_per_week_filter.csv', #was actually a 1x week filter
    header=0, index_col=None)
print('Len Kwale original sample:', len(Kwale))

sample = trt_original['0']
Nairobi_trt = Nairobi[Nairobi['xDAI_blockchain_address'].isin(sample)]
print('Len trt in Nairobi:', len(Nairobi_trt))
Kwale_trt = Kwale[Kwale['xDAI_blockchain_address'].isin(sample)]
print('Len trt in Kwale:', len(Kwale_trt))
print('Total trt size across both = ', len(Nairobi_trt) + len(Kwale_trt))

full_sample_original = pd.concat([Nairobi, Kwale])
print('len before cleaning:', len(full_sample_original))
full_sample_original = full_sample_original.loc[full_sample_original.held_roles == 'BENEFICIARY']
print('len after cleaning:', len(full_sample_original))
full_sample_original.drop(columns=['Unnamed: 0'], inplace=True)

full_sample_new = pd.read_csv('/Users/rebeccamqamelo/Desktop/Capstone/full_sample_selection_both.csv', header=0, index_col=None)
full_sample_new.drop(columns=['Unnamed: 0'], inplace=True)
full_sample_new.drop(columns=['trt_label'], inplace=True)

nairobi_sample = pd.read_csv('/Users/rebeccamqamelo/Desktop/Capstone/Rebecca_Nairobi_sample.csv', header=0, index_col=None)
nairobi_sample.drop(columns=['Unnamed: 0'], inplace=True)
nairobi_trt = nairobi_sample.loc[(nairobi_sample.trt_label == 1)]
nairobi_ctrl = nairobi_sample.loc[(nairobi_sample.trt_label == 0)]
kwale_sample = pd.read_csv('/Users/rebeccamqamelo/Desktop/Capstone/Rebecca_Kwale_sample.csv', header=0, index_col=None)
kwale_sample.drop(columns=['Unnamed: 0'], inplace=True)
kwale_trt = kwale_sample.loc[(kwale_sample.trt_label == 1)]
kwale_ctrl = kwale_sample.loc[(kwale_sample.trt_label == 0)]
#full_sample = pd.concat([nairobi_sample, kwale_sample], ignore_index=True)

full_sample_new.insert(0, 'trt_label', 0)

for i in nairobi_trt['xDAI_blockchain_address']:
    full_sample_new.loc[(full_sample_new.xDAI_blockchain_address == i), 'trt_label'] = 1
    
for i in kwale_trt['xDAI_blockchain_address']:
    full_sample_new.loc[(full_sample_new.xDAI_blockchain_address == i), 'trt_label'] = 1

print('len before cleaning:', len(full_sample_new))
full_sample_new = full_sample_new.loc[full_sample_new.held_roles == 'BENEFICIARY']
#clean_sample.drop(columns=['trt_label'], inplace=True)
full_sample_new.drop(full_sample_new[full_sample_new['gender'] == 'other'].index, inplace=True)
full_sample_new.drop(full_sample_new[full_sample_new['gender'] == 'unknown gender'].index, inplace=True)
print('len after cleaning:', len(full_sample_new))

Len Nairobi original sample: 931
Len Kwale original sample: 243
Len trt in Nairobi: 394
Len trt in Kwale: 166
Total trt size across both =  560
len before cleaning: 1174
len after cleaning: 1124
len before cleaning: 1174
len after cleaning: 979


In [89]:
def clean_genders(df):
    """
    This function removes users with genders labelled "unknown gender" or "other"
    """
    return df.loc[(df.gender != "unknown gender") & (df.gender != "other")]

def remove_extreme_outliers(df, cols):
    """
    [OPTIONAL, NOT APPLIED IN FINAL ANALYSIS]
    """
    extreme_outliers = []
    
    for col in cols:
        outliers = df.loc[(df[col] >= df[col].quantile(.99)), 'xDAI_blockchain_address']
        extreme_outliers.extend(outliers)
        
    extreme_outliers = list(set(extreme_outliers))
    print('N. extreme outliers based on {}'.format([col for col in cols]), len(extreme_outliers))
    print(extreme_outliers)
    df_no_outliers = df[~(df.xDAI_blockchain_address).isin(extreme_outliers)]
    
    return df_no_outliers
            
def remove_attrition(df):
    """
    [OPTIONAL, NOT APPLIED IN FINAL ANALYSIS]
    This function removes users who did not trade once during the 3-week intervention period.
    (Their volume traded out at the end of the three weeks will be no different from their
    volume traded out at baseline – this can be viewed as the equivalent of attrition)
    """

    trt = df.loc[(df.trt_label == 1)]
    ctrl = df.loc[(df.trt_label == 0)]
    
    # No trades in or out from before the intervention = inactive user = attrition
    ctrl_no_trade = ctrl.loc[(ctrl.svol_out_5 == ctrl.svol_out_0) & (ctrl.svol_in_5 == ctrl.svol_in_0)]['xDAI_blockchain_address'].values
    trt_no_trade = trt.loc[(trt.svol_out_5 == trt.svol_out_0) & ((trt.svol_in_5) == trt.svol_in_0 + 1200)]['xDAI_blockchain_address'].values
    no_trade = np.concatenate((ctrl_no_trade, trt_no_trade))
        
    no_trade = ctrl_no_trade
    #print('{} observations dropped due to attrition'.format(len(no_trade)))
    return df[~df.xDAI_blockchain_address.isin(no_trade)]


def assign_trt_labels(raw_df, sample):
    """
    This function takes in the trt_labels of the specifies sample set and applies it to the 
    new user dataframe, with some additional cleaning to address location names that got swapped.
    """
    df = raw_df.copy()
    df = df[df['xDAI_blockchain_address'].isin(sample['xDAI_blockchain_address'])]
    df.insert(0, 'trt_label', 0)
    for i in sample['xDAI_blockchain_address']:
        df.loc[(df.xDAI_blockchain_address == i), 'trt_label'] = int(sample.loc[(sample.xDAI_blockchain_address == i), 'trt_label'])

    # Correct erroneously swapped location names
    Nairobi = df[(df['area_name'] == 'Nairobi')]
    Kwale = df[(df['area_name'] == 'Kwale')]

    # User is in Nairobi but appears in the Kwale sample...
    swapped_N_to_K = sample[sample.xDAI_blockchain_address.isin(Kwale.xDAI_blockchain_address)]
    N_wrong_in_K = swapped_N_to_K[swapped_N_to_K.location.str.contains('Nairobi', regex=False)]
    for i in N_wrong_in_K.xDAI_blockchain_address:
        df.loc[(df.xDAI_blockchain_address == i), 'area_name'] = 'Nairobi'

    # User is in Kwale but appears in the Nairobi sample...
    swapped_K_to_N = sample[sample.xDAI_blockchain_address.isin(Nairobi.xDAI_blockchain_address)]
    K_wrong_in_N = swapped_K_to_N[swapped_K_to_N.location.str.contains('Kwale', regex=False)]
    for i in K_wrong_in_N.xDAI_blockchain_address:
        df.loc[(df.xDAI_blockchain_address == i), 'area_name'] = 'Kwale'
    
    df.loc[(df.gender == "other"), 'gender'] = "unknown gender"
    
    return df


def create_wide_panel(dfs):
    """
    This function concatenates the different timestamped dataframes to create one wide dateframe.
    """
    
    transfer = 1200/41.1
    
    new_df = pd.DataFrame([])
    new_df['xDAI_blockchain_address'] = dfs[0]['xDAI_blockchain_address']
    new_df['trt_label'] = dfs[0]['trt_label']
    new_df['area_name'] = dfs[0]['area_name']
    new_df['gender'] = dfs[0]['gender']
    new_df['biz_type'] = dfs[0]['business_type']
    new_df['days_enrolled_0'] = dfs[0]['days_enrolled']
    new_df['days_active_0'] = dfs[0]['days_active']
    new_df.insert(0, 'sex', 0)
    new_df.loc[(new_df.gender == 'female'), 'sex'] = 1

    for col in cols:
        for i in range(len(dfs)):
            new_df[col + '_{}'.format(i)] = dfs[i][col].values
            if col in PPP_cols:
                new_df[col + '_{}'.format(i)] = new_df[col + '_{}'.format(i)]/41.1 # USD/KES PPP exchange rate in 2020
            new_df.rename(columns={'females_bought_from_vol_{}'.format(i): 'fem_support_{}'.format(i)}, inplace=True)
    
    return new_df
            

def format_csvs(end_dates):
    """
    This function takes the formatted userData and saves a) a combined location csv and 
    b) a separate csv for each location, eaching containing the assigned labels for trt and ctrl units.
    """
    
    dfs_new = []
    
    for date in range(len(end_dates)):
        df = pd.read_csv('/Users/rebeccamqamelo/Desktop/Capstone/Data/userData2020-01-25-' + datetime.strftime(end_dates[date], '%Y-%m-%d') + '.csv', header=0, index_col=None)
        df_new = assign_trt_labels(df, full_sample_new)
        dfs_new.append(df_new)
                
    # Check all dfs are recorded in the group arrays
    new = create_wide_panel(dfs_new)
    #new_extreme_outliers_removed = remove_extreme_outliers(new, ['monthly_income_1'])
    print('Len new:', len(new))
    new.to_csv('/Users/rebeccamqamelo/Desktop/RCT/Data/new.csv')
    #new_extreme_outliers_removed.to_csv('/Users/rebeccamqamelo/Desktop/RCT/Data/new_extreme_outliers_removed.csv')

In [90]:
cols = ['bal', 'svol_in', 'svol_out', 'stxns_in', 'stxns_out', 'sunique_in', 'sunique_out',
        'otxns_in', 'otxns_out', 'ovol_in', 'ovol_out',
        'females_sold_to', 'females_bought_from', 'males_sold_to', 'males_bought_from', 
        'females_bought_from_vol', 'food_water_vol', 'labour_vol', 'health_vol', 'savings_vol', 
        'shop_vol', 'education_vol', 'food_water_n', 'labour_n', 'health_n', 'savings_n', 'shop_n', 
        'education_n', 'transport_n', 'energy_n', 'environment_n', 'other_n']

PPP_cols = ['bal', 'svol_in', 'svol_out', 'ovol_in', 'ovol_out', 'females_bought_from_vol', 
            'food_water_vol', 'labour_vol', 
            'health_vol', 'savings_vol', 'shop_vol', 'education_vol', 'transport_vol', 'support_net']

In [91]:
format_csvs(end_dates)

Len new: 979


In [83]:
df = pd.read_csv('/Users/rebeccamqamelo/Desktop/RCT/Data/new.csv', header=0, index_col=None)
df[(df.food_water_vol_5) < 1]

,Unnamed: 0,sex,xDAI_blockchain_address,trt_label,area_name,gender,biz_type,days_enrolled_0,days_active_0,bal_0,bal_1,bal_2,bal_3,bal_4,bal_5,bal_6,svol_in_0,svol_in_1,svol_in_2,svol_in_3,svol_in_4,svol_in_5,svol_in_6,svol_out_0,svol_out_1,svol_out_2,svol_out_3,svol_out_4,svol_out_5,svol_out_6,stxns_in_0,stxns_in_1,stxns_in_2,stxns_in_3,stxns_in_4,stxns_in_5,stxns_in_6,stxns_out_0,stxns_out_1,stxns_out_2,stxns_out_3,stxns_out_4,stxns_out_5,stxns_out_6,sunique_in_0,sunique_in_1,sunique_in_2,sunique_in_3,sunique_in_4,sunique_in_5,sunique_in_6,sunique_out_0,sunique_out_1,sunique_out_2,sunique_out_3,sunique_out_4,sunique_out_5,sunique_out_6,otxns_in_0,otxns_in_1,otxns_in_2,otxns_in_3,otxns_in_4,otxns_in_5,otxns_in_6,otxns_out_0,otxns_out_1,otxns_out_2,otxns_out_3,otxns_out_4,otxns_out_5,otxns_out_6,ovol_in_0,ovol_in_1,ovol_in_2,ovol_in_3,ovol_in_4,ovol_in_5,ovol_in_6,ovol_out_0,ovol_out_1,ovol_out_2,ovol_out_3,ovol_out_4,ovol_out_5,ovol_out_6,females_sold_to_0,females_sold_to_1,females_sold_to_2,females_sold_to_3,females_sold_to_4,females_sold_to_5,females_sold_to_6,females_bought_from_0,females_bought_from_1,females_bought_from_2,females_bought_from_3,females_bought_from_4,females_bought_from_5,females_bought_from_6,males_sold_to_0,males_sold_to_1,males_sold_to_2,males_sold_to_3,males_sold_to_4,males_sold_to_5,males_sold_to_6,males_bought_from_0,males_bought_from_1,males_bought_from_2,males_bought_from_3,males_bought_from_4,males_bought_from_5,males_bought_from_6,fem_support_0,fem_support_1,fem_support_2,fem_support_3,fem_support_4,fem_support_5,fem_support_6,food_water_vol_0,food_water_vol_1,food_water_vol_2,food_water_vol_3,food_water_vol_4,food_water_vol_5,food_water_vol_6,labour_vol_0,labour_vol_1,labour_vol_2,labour_vol_3,labour_vol_4,labour_vol_5,labour_vol_6,health_vol_0,health_vol_1,health_vol_2,health_vol_3,health_vol_4,health_vol_5,health_vol_6,savings_vol_0,savings_vol_1,savings_vol_2,savings_vol_3,savings_vol_4,savings_vol_5,savings_vol_6,shop_vol_0,shop_vol_1,shop_vol_2,shop_vol_3,shop_vol_4,shop_vol_5,shop_vol_6,education_vol_0,education_vol_1,education_vol_2,education_vol_3,education_vol_4,education_vol_5,education_vol_6,food_water_n_0,food_water_n_1,food_water_n_2,food_water_n_3,food_water_n_4,food_water_n_5,food_water_n_6,labour_n_0,labour_n_1,labour_n_2,labour_n_3,labour_n_4,labour_n_5,labour_n_6,health_n_0,health_n_1,health_n_2,health_n_3,health_n_4,health_n_5,health_n_6,savings_n_0,savings_n_1,savings_n_2,savings_n_3,savings_n_4,savings_n_5,savings_n_6,shop_n_0,shop_n_1,shop_n_2,shop_n_3,shop_n_4,shop_n_5,shop_n_6,education_n_0,education_n_1,education_n_2,education_n_3,education_n_4,education_n_5,education_n_6,transport_n_0,transport_n_1,transport_n_2,transport_n_3,transport_n_4,transport_n_5,transport_n_6,energy_n_0,energy_n_1,energy_n_2,energy_n_3,energy_n_4,energy_n_5,energy_n_6,environment_n_0,environment_n_1,environment_n_2,environment_n_3,environment_n_4,environment_n_5,environment_n_6,other_n_0,other_n_1,other_n_2,other_n_3,other_n_4,other_n_5,other_n_6,annualized_income_0,monthly_income_1,monthly_spending_1,monthly_income_5,monthly_spending_5,monthly_income_6,monthly_spending_6,MPC_5,ave_vol_out_5
0,104,0,0x338b84A3B9b99114A210f246C50A9Bd76d28E013,1,Nairobi,male,Shop,699,399,38.49,40.93,70.12,70.12,72.31,72.31,73.53,21.65,21.65,21.65,21.65,22.63,22.63,23.11,22.97,22.97,23.70,23.70,24.18,24.18,24.67,88,88,88,88,90,90,91,96,96,97,97,98,98,99,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,4,7,7,8,8,9,1,3,3,3,3,4,4,16.84,19.27,48.47,48.47,49.69,49.69,50.42,9.73,9.78,9.78,9.78,9.78,10.56,10.56,1,1,1,1,1,1,1,1,1,1,1,1,1,1,95,95,96,96,97,97,98,87,87,87,87,89,89,90,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22.97,22.97,23.70,23.70,24.18,24.18,24.67,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,96,96,97,97,98,98,99,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [64]:
ex_outliers = ['0x3b748c27b3Ac1BD64aff723A4ef987D0e1ECa668', 
               '0x8770Fe60903B26D2f04Ea4Ed50Cc20B44E7E6823', 
               '0x57c20c5C13156849089B933115ce01bc6a492A14', 
               '0x77a69CB7C6d2f8d63462A5958A27b53DF4FDeEBe', 
               '0x958385D7fcD4DB7228F0E0765D45C4d6eBD11406', 
               '0x30799915EC30A906Fe86B0a6889E834122024B8a', 
               '0x7d5FCA090DA933FCA460468B2D4aAf06ad22B99d', 
               '0x88c23f101c23b687AB601c14f1057dFe22b7785C', 
               '0xac76F62829a4972842ea939DC7F39b330520D815', 
               '0x307C6D970d02dBA3b54B92F8b98e41E0B501D30d']
for outlier in ex_outliers:
    print(userData[outlier]['area_name'])

Nairobi
Kwale
Nairobi
Nairobi
Nairobi
Kwale
Kwale
Nairobi
Nairobi
Nairobi


### Format txnData into timeseries format for analysis


Filter the transaction data to only include user wallets that are study particiapnts. Update this df with treatment and control labels for each user. This will be used to create a timeseries version of the transaction data, with transaction data for every study particant for each day e.g. for Difference in Differences analysis on Stata.

In [ ]:
userData = userData_dict.copy()
txnData = txnData_dict.copy()

In [21]:
print('Len before filter:', len(txnData))
# Filter study participants
not_in_sample = list(set(txnData.keys()).difference(full_sample.xDAI_blockchain_address))
for key in not_in_sample:
    txnData.pop(key)
print('Len after filter:', len(txnData))

Len before filter: 41731
Len after filter: 979


In [23]:
txnData_11_19 = txnData_raw[(txnData_raw.timeset >= start_date) & (txnData_raw.timeset <= end_dates[0])]
print(len(txnData_11_19))
txnData_raw.to_csv('/Users/rebeccamqamelo/Desktop/Capstone/txnData_11_19_raw_sorted.csv')

txnData_11_26 = txnData_raw[(txnData_raw.timeset >= start_date) & (txnData_raw.timeset <= end_dates[1])]
print(len(txnData_11_26))
txnData_raw.to_csv('/Users/rebeccamqamelo/Desktop/Capstone/txnData_11_26_sorted.csv')

txnData_12_03 = txnData_raw[(txnData_raw.timeset >= start_date) & (txnData_raw.timeset <= end_dates[2])]
print(len(txnData_12_03))
txnData_raw.to_csv('/Users/rebeccamqamelo/Desktop/Capstone/txnData_12_03_sorted.csv')

txnData_12_10 = txnData_raw[(txnData_raw.timeset >= start_date) & (txnData_raw.timeset <= end_dates[3])]
print(len(txnData_12_10))
txnData_raw.to_csv('/Users/rebeccamqamelo/Desktop/Capstone/txnData_12_10_raw_sorted.csv')

411373
558515
565801
572970


In [24]:
headersTxPub = ['day', 'userID', 'trt_label', 'gender', 'location', 'business_type', 'days_active', 
                'svol_in','svol_out', 'stxns_in', 'stxns_out', 'sunique_in', 'sunique_out', 
                'sunique_out_group', 'food_water', 'labour', 'health', 'savings', 'shop', 'education', 
                'transport', 'energy', 'environment', 'other', 'females_sold_to', 'females_bought_from', 
                'males_sold_to', 'males_bought_from']

In [25]:
def create_txnData(full_sample, userData, txnData, headersTxPub, start_date, end_date):

    days_int = [i for i in range((end_date - start_date).days+1)]
    days_date = [datetime.strftime(d, '%Y-%m-%d') for d in pd.date_range(start=start_date, end=end_date)]
    placeholderData = {i: 0 for i in headersTxPub[7:]} # from svol_in onwards
    new_txnDict = {user: {day: placeholderData for day in days_int} for user in txnData.keys()}
    
    for user in txnData.keys():
            
        traded_days = [list(tx.keys())[0] for tx in txnData[user]]
        all_txns = dict(ChainMap(*txnData[user]))

        svolume_in = 0
        svolume_out = 0
        stxns_out = 0
        stxns_in = 0
        sunique_txns_out = 0
        sunique_txns_out_group = 0
        sunique_txns_in = 0
        food_water = 0
        labour = 0
        health = 0
        savings = 0
        shop = 0
        education = 0
        transport = 0
        energy = 0
        environment = 0
        other = 0
        females_sold_to = 0
        males_sold_to = 0
        females_bought_from = 0
        males_bought_from = 0

        seen_days = []
            
        for date, trans in all_txns.items():

            if date in days_date: # txn falls in the window of analysis
                
                day = days_date.index(trans['timeset'])
                volume_in = 0
                volume_out = 0
                txns_in = 0 # all trades
                txns_out = 0
                unique_txns_in = 0 # unique trades
                unique_txns_out = 0
                svolume_in = 0
                svolume_out = 0
                stxns_out = 0
                stxns_in = 0
                sunique_txns_out = 0
                sunique_txns_out_group = 0
                sunique_txns_in = 0
                females_sold_to = 0
                males_sold_to = 0
                females_bought_from = 0
                males_bought_from = 0
                sseenRecUsers = [] # user's recipients ("people who've bought from me")
                seenRecUsers = []
                sseenSentUsers = [] # recipient user's partner ("people I've bought from")
                seenSentUsers = []
                bought_items = []
                sold_items = []
                food_water = 0
                labour = 0
                health = 0
                savings = 0
                shop = 0
                education = 0
                transport = 0
                energy = 0
                environment = 0
                other = 0
                                    
                if trans['source'] == user: # this txn marks where the user is the buyer
                    if trans['transfer_subtype'] == 'STANDARD':

                        svolume_out+=float(trans['weight'])
                        stxns_out+=1
                        sold_items.append(trans['transfer_use'])

                        if userData[trans['target']]['gender'] == 'female':
                            females_sold_to+=1
                        elif userData[trans['target']]['gender'] == 'male':
                            males_sold_to+=1

                        if trans['target'] not in sseenRecUsers: # txn is standard and unqiue
                            sseenRecUsers.append(trans['target'])
                            sunique_txns_out+=1
                            if userData[user]['held_roles'] == "GROUP_ACCOUNT":
                                sunique_txns_out_group += 1
                                    
                else: # this txn marks where the user was a recipient i.e. sold something
                    if trans['transfer_subtype'] == 'STANDARD':
                        svolume_in+=float(trans['weight'])
                        stxns_in+=1
                        bought_items.append(trans['transfer_use'])

                        if userData[trans['source']]['gender'] == 'female':
                            females_bought_from+=1
                        elif userData[trans['source']]['gender'] == 'male':
                            males_bought_from+=1

                        if trans['transfer_use'] == 'Food/Water':
                            food_water+=1
                        elif trans['transfer_use'] == 'Farming/Labour':
                            labour+=1
                        elif trans['transfer_use'] == 'Health':
                            health+=1
                        elif trans['transfer_use'] == 'Savings Group':
                            savings+=1
                        elif trans['transfer_use'] == 'Shop':
                            shop+=1
                        elif trans['transfer_use'] == 'Education':
                            education+=1
                        elif trans['transfer_use'] == 'Transport':
                            transport+=1
                        elif trans['transfer_use'] == 'Fuel/Energy':
                            energy+=1
                        elif trans['transfer_use'] == 'Environment':
                            environment+=1
                        elif trans['transfer_use'] == 'Other':
                            other+=1

                        if trans['source'] not in sseenSentUsers:
                            sseenSentUsers.append(trans['source'])
                            sunique_txns_in+=1
                
                # If this is not the first txn on that day, previous txn info needs to be updated
                if day in seen_days: 
                    new_txnDict[user][day]['svol_in'] += svol_in
                    new_txnDict[user][day]['svol_out'] += svol_out
                    new_txnDict[user][day]['stxns_in'] += stxns_in
                    new_txnDict[user][day]['stxns_out'] += stxns_out
                    new_txnDict[user][day]['sunique_in'] += sunique_txns_in
                    new_txnDict[user][day]['sunique_out'] += sunique_txns_out
                    new_txnDict[user][day]['sunique_out_group'] += sunique_txns_out_group
                    new_txnDict[user][day]['food_water'] += food_water
                    new_txnDict[user][day]['labour'] += labour
                    new_txnDict[user][day]['health'] += health
                    new_txnDict[user][day]['savings'] += savings
                    new_txnDict[user][day]['shop'] += shop
                    new_txnDict[user][day]['education'] += education
                    new_txnDict[user][day]['transport'] += transport
                    new_txnDict[user][day]['energy'] += energy
                    new_txnDict[user][day]['environment'] += environment
                    new_txnDict[user][day]['other'] += other
                    new_txnDict[user][day]['females_sold_to'] += females_sold_to
                    new_txnDict[user][day]['males_sold_to'] += males_sold_to
                    new_txnDict[user][day]['females_bought_from'] += females_bought_from
                    new_txnDict[user][day]['males_bought_from'] += males_bought_from
                                      
                # Else, this is the first txn on that day for this user; new info needs to be recorded
                else:
                    seen_days.append(day)
                    txData = {
                          'svol_in': svolume_in, 
                          'svol_out': svolume_out, 
                          'stxns_in':stxns_in, 
                          'stxns_out': stxns_out, 
                          'sunique_in': sunique_txns_in, 
                          'sunique_out': sunique_txns_out,
                          'sunique_out_group': sunique_txns_out_group,
                          'food_water': food_water,
                          'labour': labour,
                          'health': health,
                          'savings': savings,
                          'shop': shop,
                          'education': education,
                          'transport': transport,
                          'energy': energy,
                          'environment': environment,
                          'other': other,
                          'females_sold_to': females_sold_to,
                          'males_sold_to': males_sold_to,
                          'females_bought_from': females_bought_from,
                          'males_bought_from': males_bought_from}
                    
                    uDict = new_txnDict[user][day]
                    uDict.clear
                    uDict = txData
                    new_txnDict[user][day] = uDict             
                
    return new_txnDict

In [26]:
def generate_txn_csv(locations, txnData, userData, headersTxPub, start_date, end_date):

    headersTx = headersTxPub
    
    for loc in locations:
        filenameTx = '/Users/rebeccamqamelo/Desktop/RCT/Data/txnDataTimeSeries_' + loc + '.csv'
        print("saving all transactions to: ", filenameTx)

        timestr = time.strftime("%Y-%m-%d")
        indexR = 0
        seen_users = []
        exclude_list = []
        numberTx = 1
        numberUsers = 0

        with open(filenameTx, 'w', newline='') as csvfileTx:
            spamwriterTx = csv.writer(csvfileTx)
            spamwriterTx.writerow(headersTx)

            c_idx = 0
            chunks = 10000
            tx_hash = []

            for user, t in txnData.items():
                
                if userData[user]['area_name'] == loc:

                    for d in t: # for each day in the list of all txns
                        day = txnData[user][d]
                        row_data = {'day': d}
                        row_data['userID'] = user
                        row_data['trt_label'] = int(full_sample.loc[(full_sample.xDAI_blockchain_address == user), 'trt_label'])
                        row_data['gender'] = userData[user]['gender']
                        row_data['location'] = userData[user]['area_name']
                        row_data['business_type'] = userData[user]['business_type']
                        row_data['days_active'] = userData[user]['days_active']
                        row_data['svol_in'] = day['svol_in']
                        row_data['svol_out'] = day['svol_out']
                        row_data['stxns_in'] = day['stxns_in']
                        row_data['stxns_out'] = day['stxns_out']
                        row_data['sunique_in'] = day['sunique_in']
                        row_data['sunique_out'] = day['sunique_out']
                        row_data['sunique_out_group'] = day['sunique_out_group']
                        row_data['food_water'] = day['food_water']
                        row_data['labour'] = day['labour']
                        row_data['health'] = day['health']
                        row_data['savings'] = day['savings']
                        row_data['shop'] = day['shop']
                        row_data['education'] = day['education']
                        row_data['transport'] = day['transport']
                        row_data['energy'] = day['energy']
                        row_data['environment'] = day['environment']
                        row_data['other'] = day['other']
                        row_data['females_sold_to'] = day['females_sold_to']
                        row_data['females_bought_from'] = day['females_bought_from']
                        row_data['males_sold_to'] = day['males_sold_to']
                        row_data['males_bought_from'] = day['males_bought_from']

                        rowString = []
                        for k in headersTx:
                            if k in row_data.keys():
                                #spamwriterTx.writerow([str(row_data[k]) for k in headersTx])
                                rowString.append(str(row_data[k]))
                            else:
                                rowString.append('')

                        spamwriterTx.writerow(rowString)
                        if indexR < 3:
                            #print(row_data) #debug
                            indexR+=1
                        if c_idx >= chunks:
                            c_idx = 0
                            csvfileTx.flush()  # whenever you want
                            print("chunk: ", numberTx)
                        else:
                            c_idx += 1

                        numberTx += 1

        print("****saved all transactions to csv", filenameTx, " number of tx:", numberTx, timestr)

In [28]:
# This cell takes approx. 3 min to run!
start_date = datetime.strptime('2020-01-25', '%Y-%m-%d') # Look from the beginning of the year
#start_date = datetime.strptime('2020-09-20', '%Y-%m-%d') # Look 2 months back from RCT start date
end_date = datetime.strptime('2021-02-04', '%Y-%m-%d')

txnTimeSeries = create_txnData(full_sample, userData, txnData, headersTxPub, start_date, end_date)
generate_txn_csv(['Nairobi'], txnTimeSeries, userData, headersTxPub, start_date, end_date)

saving all transactions to:  /Users/rebeccamqamelo/Desktop/Capstone/txnDataTimeSeries_Nairobi.csv
chunk:  10001
chunk:  20002
chunk:  30003
chunk:  40004
chunk:  50005
chunk:  60006
chunk:  70007
chunk:  80008
chunk:  90009
chunk:  100010
chunk:  110011
chunk:  120012
chunk:  130013
chunk:  140014
chunk:  150015
chunk:  160016
chunk:  170017
chunk:  180018
chunk:  190019
chunk:  200020
chunk:  210021
chunk:  220022
chunk:  230023
chunk:  240024
chunk:  250025
****saved all transactions to csv /Users/rebeccamqamelo/Desktop/Capstone/txnDataTimeSeries_Nairobi.csv  number of tx: 256159 2021-01-17
saving all transactions to:  /Users/rebeccamqamelo/Desktop/Capstone/txnDataTimeSeries_Kwale.csv
chunk:  10001
chunk:  20002
chunk:  30003
chunk:  40004
chunk:  50005
****saved all transactions to csv /Users/rebeccamqamelo/Desktop/Capstone/txnDataTimeSeries_Kwale.csv  number of tx: 58102 2021-01-17


In [1]:
def traded_with(sample, txnData, start_date, end_date):
    
    sample_users = sample
    all_users = list(txnData.keys())
    interaction_df = pd.DataFrame(0, index=np.arange(len(all_users)))
    interaction_df.set_index(all_users)
    interaction_df.columns=sample_users
    days_date = [datetime.strftime(d, '%Y-%m-%d') for d in pd.date_range(start=start_date, end=end_date)]
    
    for user in all_users:
        my_trade_partners = []
        all_txns = dict(ChainMap(*txnData[user]))
        for date, trans in all_txns.items():
            if date in days_date:
                if trans['source'] == user:
                    my_trade_partners.append(trans['target'])
                else:
                    my_trade_partners.append(trans['source'])
    
        for tp in my_trade_partners:
            interaction_df.loc[tp, user]+=1
    
    return interaction_df


def sample_interactions(sample_users, full_interaction_df):
    
    return full_interaction_df[sample_users]

txnData = txnData_dict.copy()
interactions =  traded_with(full_sample_new.xDAI_blockchain_address.values,
                            txnData, 
                            datetime.strptime('2020-11-20', '%Y-%m-%d'), 
                            datetime.strptime('2021-02-04', '%Y-%m-%d'))
interactions.head()